# PostScriptML: Data Preparation and EDA Notebook
## by Dolci Key 

## Import Libraries

In [1]:
import pandas as pd
import numpy as py

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import shap

import glob
import os

from keras import models, layers
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import inception_v3, mobilenet_v2, vgg19, densenet
from keras.layers import Dense,GlobalAveragePooling2D
import tensorflow as tf
from PIL import Image, ImageOps, ImageFilter

import os, shutil
import argparse
import sys

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics.pairwise import cosine_similarity

import pickle

# Data 

### For this project, these are my own photos taken with a Nikon Z7. Photos have been pre-labeled as select and reject. I will be using an S3 Bucket to store the data as these images are .NEF files, large uncompressed images, which take up between 40-60 MB each. I used these images because professional photographers always shoot in RAW and these will be what they are looking through for post production photograph selections. 

In [ ]:
# Loading Test Train and Validation sets

In [ ]:
# Plot Class Imbalance

In [ ]:
# Plot Racial Breakdown of Images

In [2]:
# Horizontal flip image augmentation


# convert to numpy array
data = img_to_array(img)

# expand dimension to one sample
samples = expand_dims(data, 0)

# create image data augmentation generator
datagen = ImageDataGenerator(horizontal_flip=True)

# prepare iterator
it = datagen.flow(samples, batch_size=1)

# generate samples and plot
for i in range(9):
    
    # define subplot
    pyplot.subplot(330 + 1 + i)
    
    # generate batch of images
    batch = it.next()
    
    # convert to unsigned integers for viewing
    image = batch[0].astype('uint8')
    
    # plot raw pixel data
    pyplot.imshow(image)
    
# show the figure
pyplot.show()


# Code derived from original code from Machine Learning Mastery (may not use this code but putting source in case)

NameError: name 'img' is not defined